In [0]:
from torch.utils.data import Dataset, DataLoader
from torch import nn, from_numpy, optim
import numpy as np
#pandas- librărie pentru lucrul cu fișierele
import pandas as pd
import torch
import torch.nn as nn

In [2]:
from google.colab import files
uploaded = files.upload()

Saving diabetes.csv to diabetes.csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))

In [21]:
df.values

array([[-0.882353 , -0.145729 ,  0.0819672, ..., -0.766866 , -0.666667 ,
         1.       ],
       [-0.0588235,  0.839196 ,  0.0491803, ..., -0.492741 , -0.633333 ,
         0.       ],
       [-0.882353 , -0.105528 ,  0.0819672, ..., -0.923997 ,  0.       ,
         1.       ],
       ...,
       [-0.411765 ,  0.21608  ,  0.180328 , ..., -0.857387 , -0.7      ,
         1.       ],
       [-0.882353 ,  0.266332 , -0.0163934, ..., -0.768574 , -0.133333 ,
         0.       ],
       [-0.882353 , -0.0653266,  0.147541 , ..., -0.797609 , -0.933333 ,
         1.       ]])

In [0]:
#Dataset - o clasă din PyTorch foarte utilă gestionării seturilor de date
class DiabetesDataset(Dataset):
  # Initialize your data, download, etc.
  def __init__(self):
    #Citim setul de date
    df = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']), header=None, dtype=np.float32)
    xy = torch.from_numpy(df.values)
    self.len = xy.shape[0]
    #Vom folosi ca input toate valorile mai puțin ultima coloană
    self.x_data = xy[:, 0:-1]
    #Vom folosi ca output ultima coloană
    self.y_data = xy[:, [-1]]
  
  def __getitem__(self, index):
     return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len

In [0]:
dataset = DiabetesDataset()
#DataLoader - un utilitar ce ne ajută să împărțim setul de date pe batch-uri și astfel să facem antrenare în mod Mini-Batch
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=1)

In [63]:
dataset[0]

(tensor([-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333]),
 tensor([0.]))

In [0]:
class Model(nn.Module):

  def __init__(self):
    #In the constructor we instantiate two nn.Linear module
    super(Model, self).__init__()
    self.l1 = nn.Linear(8, 6)
    self.l2 = nn.Linear(6, 4)
    self.l3 = nn.Linear(4, 1)

    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    # In the forward function we accept a Variable of input data and we must return
    # a Variable of output data. We can use Modules defined in the constructor as
    # well as arbitrary operators on Variables.
    out1 = self.sigmoid(self.l1(x))
    out2 = self.sigmoid(self.l2(out1))
    y_pred = self.sigmoid(self.l3(out2))
    return y_pred
     
     

In [0]:
# Pentru o scriere și înțelegere mai ușoară a rețelei neurale putem folosi modulul nn.Sequential. Practic acesta realizează același lucru oferind o scriere 
# a codului mult mai compactă. Exemplu mai jos.

In [0]:
class Model(nn.Module):
  def __init__(self):
    # Dat fiind faptul că 
    super(Model, self).__init__()
    
    #Practic toate functiile liniare și non-liniare realizează o secvență a flowlui de date
    self.sequential=nn.Sequential(
         nn.Linear(8,6),
         nn.ReLU(),
         nn.Linear(6,4),
         nn.ReLU(),
         nn.Linear(4,1),
         nn.Sigmoid()
    )

    def forward(self, x):
      return self.sequential(x)
        

In [0]:
model=Model()

In [0]:
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

In [75]:
# Training loop
for epoch in range(200):
  # Aplicăm modificarea de learning rate per epocă, deci va trebui să avem un istoric al loss-ului
  losses=[]

  for i, data in enumerate(train_loader, 0):
    # get the inputs
    inputs, labels = data

    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(inputs)

    # Compute and print loss
    loss = criterion(y_pred, labels)
    print(f'Epoch {epoch + 1} | Batch: {i+1} | Loss: {loss.item():.4f}')

    losses.append(loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    mean_loss=sum(losses)/len(losses)
    scheduler.step(mean_loss)

Epoch 1 | Batch: 1 | Loss: 19.5965
Epoch 1 | Batch: 2 | Loss: 18.0275
Epoch 1 | Batch: 3 | Loss: 21.6898
Epoch 1 | Batch: 4 | Loss: 20.1102
Epoch 1 | Batch: 5 | Loss: 21.1660
Epoch 1 | Batch: 6 | Loss: 20.1249
Epoch 1 | Batch: 7 | Loss: 19.0796
Epoch 1 | Batch: 8 | Loss: 22.2254
Epoch 1 | Batch: 9 | Loss: 20.6432
Epoch 1 | Batch: 10 | Loss: 23.7736
Epoch 1 | Batch: 11 | Loss: 19.0665
Epoch 1 | Batch: 12 | Loss: 21.1565
Epoch 1 | Batch: 13 | Loss: 20.6419
Epoch 1 | Batch: 14 | Loss: 21.7001
Epoch 1 | Batch: 15 | Loss: 19.5921
Epoch 1 | Batch: 16 | Loss: 20.6356
Epoch 1 | Batch: 17 | Loss: 21.6910
Epoch 1 | Batch: 18 | Loss: 19.0689
Epoch 1 | Batch: 19 | Loss: 20.6417
Epoch 1 | Batch: 20 | Loss: 21.1750
Epoch 1 | Batch: 21 | Loss: 21.1714
Epoch 1 | Batch: 22 | Loss: 19.5810
Epoch 1 | Batch: 23 | Loss: 21.6843
Epoch 1 | Batch: 24 | Loss: 16.4621
Epoch 2 | Batch: 1 | Loss: 21.6859
Epoch 2 | Batch: 2 | Loss: 19.5987
Epoch 2 | Batch: 3 | Loss: 22.7298
Epoch 2 | Batch: 4 | Loss: 21.1763
Epoch